### Load environment

In [1]:
from ultralytics import YOLO
from data.json_to_yolo import Json2YoloAnnConverter
from data.xml_to_json import jsonAnnGtor
from engine.da_eval import eval_yolo, eval_da_yolo, eval_plot_COCO
import os

criteria = ['handstate', 'handside', 'hand+obj', None]        # handstate handside hand+obj all
criterion = criteria[3]   # change in case needed

/home/swdev/contactEst/cfe/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Preparing dataset in required architecture

#### Remap the original `xml` format annotation into `json`

In [2]:
prefix = '/home/swdev/contactEst/InteractionDetectorDDETR/'
datalistPath = f"{prefix}pascal_voc_format/VOCdevkit2007_handobj_100K/VOC2007/ImageSets/Main"
annPath = f"{prefix}pascal_voc_format/VOCdevkit2007_handobj_100K/VOC2007/Annotations"
targetPath = "./data/json_format"
if not os.path.exists(targetPath):
    os.makedirs(targetPath)
jsonAnnGtor(datalistPath=datalistPath, annPath=annPath, targetPath=targetPath)

### Annotation remapping for `yolo` from `json` to `txt`

In [3]:
ann_path = targetPath
save_path = './data/100DOH/labels'
Y_Cter = Json2YoloAnnConverter(ann_path=ann_path, save_path=save_path)
Y_Cter(constraint=criterion)

### Generate COCO formated ground truth in `json`
default path: `"./data/json_format/coco_gt"` 

### Load and Fine tune `YOLOv8` on Dataset `100DOH`

In [ ]:
mdl = '/home/swdev/contactEst/InteractionDetectorDDETR/ddetr_dv/models/yolo_backbone/best_l.pt'     # Modify your own model path
train_yolo = False
model = YOLO(mdl, task='detect').cuda()
nParams = sum([p.numel() for p in model.parameters()])
print(f'The model contains {nParams/1e6:.1f}M parameters!')
if train_yolo:
    data_yaml = '/home/swdev/contactEst/InteractionDetectorDDETR/ddetr_dv/data/100DOH/DOH_all.yml'
    model.train(data=data_yaml, epochs=10, imgsz=480, batch=12, save_period=1, lr0=1e-6, lrf=1e-2)

### Evaluate yolo on `100DOH` testset

In [ ]:
# Select a criterion to evaluate
criterion = criteria[0]     # ['handstate', 'handside', 'hand+obj', None]
is_demo = True

fp = eval_yolo(model, criterion, is_demo=is_demo)
eval_plot_COCO(model, fp, criterion)

#### Plots

In [ ]:
criterion = criteria[2]      # Choose one of all generated detection results to plot
eval_plot_COCO(model, fp, criterion)

### Evalaute Domain Adaptabiilty

Evaluate the domain adaptability of the loaded YOLO model on the custom dataset

In [ ]:
eval_da_yolo(model, criterion)